Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (10000, 28, 28), (10000,))


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 784), (200000, 10))
('Validation set', (10000, 784), (10000, 10))
('Test set', (10000, 784), (10000, 10))


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 0.01*tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [22]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 47.304199
Training accuracy: 10.8%
Validation accuracy: 13.5%
Loss at step 100: 11.832731
Training accuracy: 73.7%
Validation accuracy: 72.8%
Loss at step 200: 4.476631
Training accuracy: 79.3%
Validation accuracy: 77.2%
Loss at step 300: 1.979096
Training accuracy: 82.3%
Validation accuracy: 80.0%
Loss at step 400: 1.130128
Training accuracy: 83.8%
Validation accuracy: 81.5%
Loss at step 500: 0.837853
Training accuracy: 84.3%
Validation accuracy: 82.0%
Loss at step 600: 0.736061
Training accuracy: 84.3%
Validation accuracy: 82.2%
Loss at step 700: 0.700307
Training accuracy: 84.5%
Validation accuracy: 82.2%
Loss at step 800: 0.687669
Training accuracy: 84.4%
Validation accuracy: 82.2%
Test accuracy: 88.5%


In [7]:
batch_size = 128

graph_relu = tf.Graph()
with graph_relu.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
    
  
  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  hidden = tf.nn.relu(logits1)
  logits2 = tf.matmul(hidden, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)) + \
    + 0.002*tf.nn.l2_loss(weights1) \
    + 0.002*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_prediction1, weights2) + biases2)
  test_prediction1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(
    tf.matmul(test_prediction1, weights2) + biases2)
    

In [9]:
num_steps = 3001

with tf.Session(graph=graph_relu) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss at step 0: 951.844177
Minibatch accuracy: 7.8%
Validation accuracy: 29.1%
Minibatch loss at step 500: 231.251831
Minibatch accuracy: 73.4%
Validation accuracy: 82.0%
Minibatch loss at step 1000: 83.372635
Minibatch accuracy: 86.7%
Validation accuracy: 83.4%
Minibatch loss at step 1500: 30.945084
Minibatch accuracy: 82.0%
Validation accuracy: 85.9%
Minibatch loss at step 2000: 11.577959
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 2500: 4.624061
Minibatch accuracy: 91.4%
Validation accuracy: 87.5%
Minibatch loss at step 3000: 2.069727
Minibatch accuracy: 88.3%
Validation accuracy: 87.4%
Test accuracy: 92.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [105]:
batch_size = 128

graph_relu = tf.Graph()
with graph_relu.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 200]))
  biases1 = tf.Variable(tf.zeros([200]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([200, 50]))
  biases2 = tf.Variable(tf.zeros([50]))
    
  weights3 = tf.Variable(
    tf.truncated_normal([50, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))
    
  
  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  hidden = tf.nn.relu(logits1)
  logits2 = tf.matmul(hidden, weights2) + biases2
  hidden2 = tf.nn.relu(logits2)
  logits3 = tf.matmul(hidden2, weights3) + biases3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits3, tf_train_labels)) + \
    + 0.001*tf.nn.l2_loss(weights1) \
    + 0.001*tf.nn.l2_loss(weights2) \
    + 0.001*tf.nn.l2_loss(weights3) \
  

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

  #global_step = tf.Variable(0)  # count the number of steps taken.
  #learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.8)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits3)
  valid_prediction1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction2 = tf.nn.relu(tf.matmul(valid_prediction1, weights2) + biases2)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction2, weights3) + biases3)
  test_prediction1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction2 = tf.nn.relu(tf.matmul(test_prediction1, weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(test_prediction2, weights3) + biases3)
    

In [106]:
num_steps = 100000

with tf.Session(graph=graph_relu) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if step % 10 == 0:
        print 'loss = ', l
        
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    

Initialized
loss =  551.71
Minibatch loss at step 0: 551.709900
Minibatch accuracy: 7.0%
Validation accuracy: 10.6%
loss =  99.8806
loss =  98.9292
loss =  98.8434
loss =  98.2408
loss =  97.736
loss =  97.4465
loss =  97.0588
loss =  98.6389
loss =  96.4091
loss =  96.0047
loss =  95.584
loss =  95.298
loss =  94.9393
loss =  94.4616
loss =  94.0863
loss =  93.857
loss =  93.433
loss =  93.0614
loss =  92.6503
loss =  92.356
loss =  91.8673
loss =  91.6079
loss =  91.215
loss =  90.8931
loss =  90.5162
loss =  90.4295
loss =  89.7708
loss =  89.4817
loss =  89.128
loss =  88.7574
loss =  88.4636
loss =  88.0977
loss =  87.646
loss =  87.2991
loss =  86.9824
loss =  86.7312
loss =  86.3237
loss =  85.9761
loss =  85.6617
loss =  85.3093
loss =  84.9841
loss =  84.6826
loss =  84.3233
loss =  83.9562
loss =  83.6765
loss =  83.3466
loss =  83.0161
loss =  83.0728
loss =  82.3031
loss =  82.0566
Minibatch loss at step 500: 82.056610
Minibatch accuracy: 12.5%
Validation accuracy: 16.4%
lo

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
